In [ ]:
# @title 🌍 Earth Engine Noncommercial EECU Monitor
# @markdown ### **Instructions**
# @markdown 1. Enter your **Project ID**.
# @markdown 2. Select **User Tier** (Optional: Select "None" for raw usage, or a tier for quota comparisons).
# @markdown 3. Run the cell to check your status.

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import calendar
from datetime import datetime, timedelta, timezone
from google.colab import auth
from google.cloud import monitoring_v3
from IPython.display import display, Markdown

# --- USER CONFIGURATION ---
PROJECT_ID = "ee-yourproject" # @param {type:"string"}
SHOW_QUOTA_TIER = "Community" # @param ["None", "Community", "Contributor", "Partner"]
MONTHS_TO_SHOW = 12 # @param {type:"slider", min:1, max:12, step:1}

# Define limits based on tier
TIER_LIMITS = {
    "Community": 150,
    "Contributor": 1000,
    "Partner": 100000
}
QUOTA_LIMIT = TIER_LIMITS.get(SHOW_QUOTA_TIER)

# Authenticate
try:
    auth.authenticate_user()
except:
    pass

def get_eecu_data(project_id, months_back):
    if not project_id or project_id == "YOUR_PROJECT_ID_HERE":
        return pd.DataFrame()

    client = monitoring_v3.MetricServiceClient()
    project_name = f"projects/{project_id}"

    # --- 1. DEFINE TIME WINDOW (CALENDAR ALIGNED) ---
    now = datetime.now(timezone.utc)

    # Calculate start date (1st of the target month)
    # Logic: If months_back=1, we want start date = 1st of current month.
    # So we subtract (months_back - 1).
    start_year = now.year
    start_month = now.month - (months_back - 1)

    while start_month <= 0:
        start_month += 12
        start_year -= 1
    start_time = datetime(start_year, start_month, 1, tzinfo=timezone.utc)

    interval = monitoring_v3.TimeInterval({
        "end_time": {"seconds": int(now.timestamp()), "nanos": 0},
        "start_time": {"seconds": int(start_time.timestamp()), "nanos": 0}
    })

    try:
        results = client.list_time_series(
            request={
                "name": project_name,
                "filter": 'metric.type = "earthengine.googleapis.com/project/cpu/usage_time"',
                "interval": interval,
                "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
                "aggregation": {
                    "alignment_period": {"seconds": 86400},
                    "per_series_aligner": monitoring_v3.Aggregation.Aligner.ALIGN_SUM,
                    "cross_series_reducer": monitoring_v3.Aggregation.Reducer.REDUCE_SUM
                }
            }
        )
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

    data = []
    for result in results:
        for point in result.points:
            data.append({
                "date": point.interval.end_time,
                "daily_hours": point.value.double_value / 3600
            })

    if not data:
        return pd.DataFrame(columns=["date", "daily_hours"])

    df = pd.DataFrame(data)

    # --- 2. DATA PROCESSING ---
    df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None).dt.normalize()
    df = df.groupby("date")["daily_hours"].sum().to_frame()

    # --- 3. FILL GAPS ---
    last_day_of_month = calendar.monthrange(now.year, now.month)[1]
    end_of_month_date = datetime(now.year, now.month, last_day_of_month)

    full_idx = pd.date_range(
        start=start_time.replace(tzinfo=None).date(),
        end=end_of_month_date.date(),
        freq='D'
    )
    df = df.reindex(full_idx, fill_value=0)
    df.index.name = 'date'

    # --- 4. CUMULATIVE MATH ---
    df["month_group"] = df.index.to_period('M')
    df["monthly_cumulative"] = df.groupby("month_group")["daily_hours"].cumsum()

    return df

# --- OUTPUT GENERATION ---
if PROJECT_ID != "YOUR_PROJECT_ID_HERE":
    df = get_eecu_data(PROJECT_ID, MONTHS_TO_SHOW)

    if not df.empty and 'daily_hours' in df.columns:

        # --- STATUS CHECK ---
        current_date = pd.Timestamp.today().normalize()
        if current_date in df.index:
            current_val = df.loc[current_date, "monthly_cumulative"]
        else:
            current_val = df["monthly_cumulative"].iloc[-1]

        if QUOTA_LIMIT:
            if current_val < QUOTA_LIMIT:
                remaining = QUOTA_LIMIT - current_val
                status_msg = f"**✅ Within {SHOW_QUOTA_TIER} Tier** ({current_val:.1f} / {QUOTA_LIMIT} hrs used). You have **{remaining:.1f} hours** remaining."
            else:
                status_msg = f"**⛔ Quota Depleted** ({current_val:.1f} hrs used). You have exceeded the {QUOTA_LIMIT}hr {SHOW_QUOTA_TIER} limit."
        else:
            status_msg = f"**{current_val:.1f} hours** used this month."

        display(Markdown(f"### Current Month Status: {status_msg}"))

        # --- PLOTLY CHART ---
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # A. Bar chart (daily)
        fig.add_trace(
            go.Bar(
                x=df.index,
                y=df['daily_hours'],
                name="Daily Usage",
                marker_color='#4c78a8',
                opacity=0.7,
                hovertemplate='%{x|%b %d}<br>Daily: %{y:.2f} hrs<extra></extra>'
            ),
            secondary_y=False,
        )

        # B. Line chart (cumulative)
        fig.add_trace(
            go.Scatter(
                x=df.index,
                y=df['monthly_cumulative'],
                name="Month Total",
                mode='lines',
                line=dict(color='#ff7f0e', width=3),
                hovertemplate='%{x|%b %d}<br>Month Total: %{y:.2f} hrs<extra></extra>'
            ),
            secondary_y=True,
        )

        # C. month separators
        month_starts = df.resample('MS').first().index
        for date in month_starts:
            fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="gray", opacity=0.3)

        # D. Quota guideline (based on selection)
        max_monthly = df['monthly_cumulative'].max()
        range_max = max_monthly * 1.1

        if QUOTA_LIMIT:
            fig.add_hline(
                y=QUOTA_LIMIT, line_dash="dot", line_color="orange", secondary_y=True,
            )
            # Ensure limit is visible if usage is low
            range_max = max(range_max, QUOTA_LIMIT * 1.05)

        # Layout
        fig.update_layout(
            title_text=f"Earth Engine EECU Usage (Last {MONTHS_TO_SHOW} Months)",
            hovermode="x unified",
            showlegend=False,
            height=500,
            bargap=0.1,
            margin=dict(l=20, r=20, t=50, b=20)
        )

        max_daily = df['daily_hours'].max()
        if pd.isna(max_daily) or max_daily == 0: max_daily = 1

        fig.update_yaxes(title_text="Daily Usage (EECU-Hours)", color='#4c78a8', secondary_y=False, showgrid=False, range=[0, max_daily * 1.1])
        fig.update_yaxes(title_text="Cumulative Monthly Total (EECU-Hours)<br>(resets to zero on the first of each month)", color='#ff7f0e', secondary_y=True, showgrid=False, range=[0, range_max])

        fig.show()

        # --- SUMMARY TABLE ---
        summary = df.resample('ME')['daily_hours'].agg(['sum', 'mean', 'max']).round(2)
        summary.columns = ['Total EECU-Hours', 'Avg Daily', 'Peak Day']
        summary.index = summary.index.strftime('%B %Y')
        summary = summary.reset_index().rename(columns={'date': 'Month'})

        # Display as a clean table
        print("\n")
        display(summary.style.format({
            'Total Hours': '{:.2f}',
            'Avg Daily': '{:.2f}',
            'Peak Day': '{:.2f}'}).hide(axis='index'))

    else:
        print("No data found. Check your Project ID.")
else:
    print("Please enter a valid Project ID above.")